![Factories creating emissions](pollution.jpg)

Photo by Maxim Tolchinskiy on Unsplash
  
When factoring heat generation required for the manufacturing and transportation of products, _Greenhouse gas emissions attributable to products, from food to sneakers to appliances, make up more than 75% of global emissions._ (`Source: The Carbon Catalogue https://www.nature.com/articles/s41597-022-01178-9`)

Our data, which is publicly available on nature.com, contains product carbon footprints (PCFs) for various companies. PCFs are the greenhouse gas emissions attributable to a given product, measured in CO<sub>2</sub> (carbon dioxide equivalent).
<!--https://www.nature.com/articles/s41597-022-01178-9-->

This data is stored in a PostgreSQL database containing one table, `product_emissions`, which looks at PCFs by product as well as the stage of production that these emissions occurred. Here's a snapshot of what `product_emissions` contains in each column:

### `product_emissions`

| field                              | data type |
|------------------------------------|-----------|
| `id`                                 | `VARCHAR`   |
| `year`                               | `INT`       |
| `product_name`                       | `VARCHAR`   |
| `company`                            | `VARCHAR`   |
| `country`                            | `VARCHAR`   |
| `industry_group`                     | `VARCHAR`   |
| `weight_kg`                          | `NUMERIC`   |
| `carbon_footprint_pcf`               | `NUMERIC`   |
| `upstream_percent_total_pcf`         | `VARCHAR`   |
| `operations_percent_total_pcf`       | `VARCHAR`   |
| `downstream_percent_total_pcf`       | `VARCHAR`   |

You'll use this data to examine the carbon footprint of each industry in the dataset! 

If you want to use a more advanced approach, you can use:

1. common table expressions (CTEs),
2. window functions, and
3. subqueries.

In [26]:
-- Goal: For the most recent year in the table, return (1) industry group,
-- (2) number of unique companies, and (3) total carbon footprint (rounded),
-- sorted from highest to lowest total footprint.

-- Create a CTE that adds a "recency rank" for each row
WITH product_emissions_filtered AS (
  	SELECT
    	industry_group,            
    	company,                   
    	carbon_footprint_pcf,      
    	year,                      
    	-- Rank rows by year, newest year first.
    	-- All rows from the most recent year will receive rank = 1.
    	RANK() OVER (ORDER BY year DESC) AS year_new
	FROM product_emissions
)

-- Aggregate results for only the most recent year
SELECT
	pef.industry_group,
	-- Count unique companies in each industry group
	COUNT(DISTINCT pef.company) AS num_companies,
	-- Sum total carbon footprint for the industry group and round to 1 decimal place
	ROUND(SUM(pef.carbon_footprint_pcf), 1) AS total_industry_footprint
FROM (
	-- Filter the CTE to keep only records from the most recent year (rank = 1)
	SELECT *
	FROM product_emissions_filtered
	WHERE year_new = 1
) AS pef
-- Aggregate per industry group
GROUP BY industry_group
-- Sort industries by total footprint (largest first)
ORDER BY total_industry_footprint DESC;

,industry_group,num_companies,total_industry_footprint
0,Materials,3,107129.0
1,Capital Goods,2,94942.7
2,Technology Hardware & Equipment,4,21865.1
3,"Food, Beverage & Tobacco",1,3161.5
4,Commercial & Professional Services,1,740.6
5,Software & Services,1,690.0


Or, if you want a more straightforward solution.

In [27]:
-- Goal: For the most recent year in the table, return (1) industry group,
-- (2) number of unique companies, and (3) total carbon footprint (rounded),
-- sorted from highest to lowest total footprint.
SELECT
	pe.industry_group,   
	-- Count unique companies in each industry group
	COUNT(DISTINCT pe.company) AS num_companies,

	-- Total carbon footprint for the industry (latest year), rounded to 1 decimal place
	ROUND(SUM(pe.carbon_footprint_pcf), 1) AS total_industry_footprint
FROM product_emissions AS pe
-- Keep only rows from the most recent year in the entire database
WHERE pe.year = (SELECT MAX(year) FROM product_emissions)
-- Aggregate per industry group
GROUP BY pe.industry_group
-- Sort industries by total footprint (largest first)
ORDER BY total_industry_footprint DESC;

,industry_group,num_companies,total_industry_footprint
0,Materials,3,107129.0
1,Capital Goods,2,94942.7
2,Technology Hardware & Equipment,4,21865.1
3,"Food, Beverage & Tobacco",1,3161.5
4,Commercial & Professional Services,1,740.6
5,Software & Services,1,690.0
